# NVCL Dataservices Notebook - Get Datasets

This notebook demonstrates on how you can query NVCL Dataservices to obtain the list of datasets available

Webserver URL of different states:

https://nvclwebservices.csiro.au/geoserver/wfs <br/>
https://gs.geoscience.nsw.gov.au/geoserver/wfs <br/>
https://geology.information.qld.gov.au/geoserver/wfs <br/>
https://sarigdata.pir.sa.gov.au/geoserver/wfs <br/>
http://geossdi.dmp.wa.gov.au/services/wfs <br/>
https://www.mrt.tas.gov.au/web-services/wfs <br/>
http://geology.data.nt.gov.au/geoserver/wfs <br/>
http://geology.data.vic.gov.au/nvcl/wfs <br/>

In [17]:
from nvcl_kit.reader import NVCLReader 
from types import SimpleNamespace
param = SimpleNamespace()

param.WFS_URL = "http://geology.data.vic.gov.au/nvcl/wfs"

# URL of NVCL service
param.NVCL_URL = "http://geology.data.vic.gov.au/NVCLDataServices/"

reader = NVCLReader(param)

In [18]:
if not reader.wfs:
    print("ERROR!")

In [19]:
nvcl_id_list = reader.get_nvcl_id_list()

In [20]:
import pandas as pd
import xmltodict
import json

data = []
for nvcl_id in nvcl_id_list:
  response_str = reader.svc.get_dataset_collection(nvcl_id)
  dataset_obj = xmltodict.parse(response_str)["DatasetCollection"]['Dataset']
  for dataset_dict in (dataset_obj if type(dataset_obj) is list else [dataset_obj]):
    dataset_dict['nvclID'] = nvcl_id
    data.append(dataset_dict)

dataset_df = pd.DataFrame(data)

In [1]:
import jupyterlab_dash
import dash
import dash_html_components as html

viewer = jupyterlab_dash.AppViewer()

app = dash.Dash(__name__)

app.layout = html.Div('Hello World')

viewer.show(app)

In [22]:
from ipywidgets import Dropdown, Layout, HTML
from IPython.display import clear_output

dataset_dw = Dropdown(
    options = list(zip(dataset_df['DatasetName'], dataset_df['DatasetID'])),
    value = dataset_df['DatasetID'][0],
    description = 'Dataset',
    layout = Layout(margin='20px')
)

imglogs = reader.get_tray_thumb_imglogs(dataset_df['DatasetID'][0])
mosaic_html = reader.get_mosaic_image(imglogs[0].log_id)

def on_change(change):
  if change['type'] == 'change' and change['name'] == 'value':
    imglogs = reader.get_tray_thumb_imglogs(change['new'])
    mosaic_html = reader.get_mosaic_image(imglogs[0].log_id)
    clear_output(True)
    display(dataset_w, HTML(value=mosaic_html))

dataset_dw.observe(on_change)

display(dataset_dw, HTML(value=mosaic_html))

Dropdown(description='Dataset', layout=Layout(margin='20px'), options=(('877270_KIND168', 'c7bfb49c-2e85-4d31-…

HTML(value='\r\n\r\n\r\n\r\n\r\n\r\n<!DOCTYPE html>\r\n<html>\r\n  <head>\r\n    <title>NVCL Data Services :: …